In [ ]:
import numpy as np
import pandas as pd
import platform
import os
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
data_direc = os.getcwd() + "/"
if platform.system() == "Windows":
    data_direc.replace("/", "\\")
train = pd.read_csv(data_direc + "gisette_train.csv")
train_labels = pd.read_csv(data_direc + "gisette_train_labels.csv")
test = pd.read_csv(data_direc + "gisette_valid.csv")
test_labels = pd.read_csv(data_direc + "gisette_valid_labels.csv")
train = np.delete(train, 5000, axis=1)
test = np.delete(test, 5000, axis=1)

train_labels_array = (train_labels.squeeze() + 1) / 2

In [26]:
from scipy.optimize import minimize

# Define Lorenz loss function
def lorenz_loss(beta, train, labels, mu):
    z = np.dot(train, beta)
    loss = np.sum(np.log(1+np.exp(-labels.T*z))) + 0.5*mu*np.linalg.norm(beta)**2
    return loss


In [27]:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

def fsa(train, train_labels, test, test_labels, k):
    s=.001
    mu = 300
    max_iter = 300
    beta = np.zeros(train.shape[1])
    losses = []

    for i in range(max_iter):
        # define the loss functio wrt beta
        loss_function = lambda beta: lorenz_loss(beta, train, train_labels, mu)

        # minimize the loss function to update beta
        result = minimize(loss_function, beta, method='L-BFGS-B')
        beta = result.x

        #compute the loss
        loss = lorenz_loss(beta, train, train_labels, mu)
        losses.append(loss)
    
    # select top k features
    feature_indices = np.argsort(np.abs(beta))[-k:]

    # train logistic regression with selected features
    lr = LogisticRegression(max_iter)
    lr.fit(train[:, feature_indices], train_labels)
    
    # compute misclassification error
    train_error = 1 - accuracy_score(train_labels, lr.predict(train[:, feature_indices]))
    test_error = 1 - accuracy_score(test_labels, lr.predict(test[:, feature_indices]))

    return train_error, test_error, losses

In [28]:
beta = np.zeros(train.shape[1])
mu = 300

# define the loss function wrt beta
loss_function = lambda beta: lorenz_loss(beta, train, train_labels, mu)

# minimize the loss function to update beta
result = minimize(loss_function, beta, method='L-BFGS-B')

/opt/miniconda3/envs/data_analysis/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


ValueError: The user-provided objective function must return a scalar value.

In [29]:
z = np.dot(train, beta)
loss = np.sum(np.log(1+np.exp(-train_labels.T*z))) + 0.5*mu*np.linalg.norm(beta)**2

/opt/miniconda3/envs/data_analysis/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0       0.693147
1       0.693147
2       0.693147
3       0.693147
4       0.693147
          ...   
5995    0.693147
5996    0.693147
5997    0.693147
5998    0.693147
5999    0.693147
Length: 6000, dtype: float64

In [ ]:
# use different k values
k_values = [10, 30, 100, 300, 500]
train_errors = []
test_errors = []
train_losses = []

# train the FSA classifier for different k values
for k in k_values:
    train_error, test_error, losses = fsa(train, train_labels, test, test_labels, k)
    train_errors.append(train_error)
    test_errors.append(test_errors)
    train_losses.append(losses)

In [ ]:
# plot training loss vs iteration number for k=100
plt.plot(range(300), train_losses[2]) #index 2 for k=100
plt.title("Training Loss vs Iteration Number for k = 100")
plt.xlabel("Iteration")
plt.ylabel("Training Loss")
plt.grid(True)
plt.show()


In [ ]:
# report misclassification errors in a table
print("k\tTrain Error\tTest Error")
for i, k in enumerate(k_values):
    print(f"{k}\t{train_errors[i]:.4f}\t\t{test_errors[i]:.4f}")
    

In [ ]:
# plot misclassification error vs k
plt.plot(k_values, train_errors, label = "Train Error")
plt.plot(k_values, test_errors, label = "Test Error")
plt.title("Misclassification Error vs Number of Features")
plt.xlabel("Number of Features (k)")
plt.ylabel("Misclassification Error")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# plot the ROC curves for k=100
k_100_features = np.argsort(np.abs(beta))[-100:]
lr_100_features = LogisticRegression(max_iter)
lr_100_features.fit(train[:, k_100_features], train_labels)
train_labels_pred = lr_100_features.predict_proba(train[:, k_100_features])[:,1]
test_labels_pred = lr_100_features.predict_proba(test[:, k_100_features])[:,1]

train_false_pos_rate, train_true_pos_rate, _ = roc_curve(train_labels, train_labels_pred)
test_false_pos_rate, test_true_pos_rate, _ = roc_curve(test_labels, test_labels_pred)

roc_auc_train = auc(train_false_pos_rate, train_true_pos_rate)
roc_auc_test = auc(test_false_pos_rate, test_true_pos_rate)

In [ ]:

plt.plot(train_false_pos_rate, train_true_pos_rate,
    label="Training ROC Curve (AUC = {:.2f})".format(roc_auc_train))
plt.plot(false_pos_test, true_pos_test, label="Test ROC Curve (AUC = {:.2f})".format(roc_auc_test))
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()